In [5]:
import pandas as pd
import os
import glob

# --- Bước 1: Thư mục chứa file CSV ---
folder_path = r"D:\Momo_shopee_normalization\MOMO_DATA_shopee\Raw\Dữ liệu Orders_shopee\Order_cancel_completed"  # thay bằng đường dẫn folder của bạn

# --- Bước 2: Lấy danh sách tất cả file CSV trong folder ---
all_files = glob.glob(os.path.join(folder_path, "*.xlsx"))

# --- Bước 3: Đọc và ghép tất cả file ---
dfs = []
for file in all_files:
    df = pd.read_excel(file)
    dfs.append(df)

merged_df = pd.concat(dfs, ignore_index=True)

# --- Bước 4: Chuẩn hóa cột thời gian và sắp xếp ---
time_col = 'Ngày đặt hàng'  # đổi tên cột thời gian nếu khác
merged_df[time_col] = pd.to_datetime(merged_df[time_col], errors='coerce')
merged_df = merged_df.sort_values(by=time_col)

# --- Bước 5: Loại bỏ trùng lặp ---
merged_df = merged_df.drop_duplicates()

# --- Bước 6: Lưu file Parquet ---
output_file = os.path.join(folder_path, "merged_order_all.parquet")
merged_df.to_parquet(output_file, index=False)
print(f"✅ File Parquet đã được lưu: {output_file}")


✅ File Parquet đã được lưu: D:\Momo_shopee_normalization\MOMO_DATA_shopee\Raw\Dữ liệu Orders_shopee\Order_cancel_completed\merged_order_all.parquet


In [9]:
import pandas as pd

In [10]:
df = pd.read_parquet(r"D:\Momo_shopee_normalization\MOMO_DATA_shopee\Raw\Dữ liệu Orders_shopee\Order_cancel_completed\merged_order_all.parquet")

In [ ]:
df

In [ ]:
import pandas as pd

# 1. Load dữ liệu
df = pd.read_parquet(r"D:\Momo_shopee_normalization\MOMO_DATA_shopee\Raw\Dữ liệu Orders_shopee\Order_cancel_completed\merged_order_all.parquet")

# 2. Định nghĩa từ khóa cho từng dòng sản phẩm
categories = {
    "tã bỉm dán momii": ["tã bỉm dán momii"],
    "tã bỉm quần momii": ["tã bỉm quần momii"],
    "tã bỉm quần momoji air dry": ["tã bỉm quần momoji air dry"],
    "tã bỉm dán momoji air dry": ["tã bỉm dán momoji air dry"],
    "tã bỉm dán momoji": ["tã bỉm dán momoji"],
    "tã bỉm quần momoji": ["tã bỉm quần momoji"],
    "băng vệ sinh": ["băng vệ sinh", "bvs"],
    "thanh chắn giường gia đình văn hóa": ["thanh chắn giường gia đình văn hóa"],
    "thảm xốp": ["thảm xốp"]
}

# 3. Hàm xác định dòng sản phẩm
def map_category(product_name):
    name = str(product_name).lower()
    for cat, kw_list in categories.items():
        if any(kw in name for kw in kw_list):
            return cat
    return "Khác"

# 4. Thêm cột "Dòng sản phẩm"
df["Dòng sản phẩm"] = df["Tên sản phẩm"].apply(map_category)

# 5. Kiểm tra kết quả
print(df[["Tên sản phẩm", "Dòng sản phẩm"]].head(20))
